In [39]:
model_id = "multi-qa-distilbert-cos-v1"

In [40]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_id)

/home/yusuf/.local/share/virtualenvs/llm-datatalks-club-OTlqznPF/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [41]:
user_question = "I just discovered the course. Can I still join it?"


In [42]:
embedding_model.encode(user_question)[0]

0.07822264

## Q2

In [43]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [44]:
docs_response.json()[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [45]:
documents = [entry for entry in documents if entry['course'] == 'machine-learning-zoomcamp']

In [46]:
len(documents)

375

In [47]:
import numpy as np

In [48]:
embeddings = []
i=0
for doc in documents:
   qa_text = f"{doc['question']} {doc['text']}"
   emb = embedding_model.encode(qa_text)
   embeddings.append(emb)

In [49]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [50]:
import numpy as np

In [51]:
len(embeddings)

375

In [52]:
X = np.array(embeddings)

In [53]:
X.shape

(375, 768)

In [54]:
user_vector = embedding_model.encode(user_question)

In [55]:
scores = X.dot(user_vector)
scores.max()

0.6506573

## Q4

In [56]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [57]:
from tqdm.auto import tqdm

In [58]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [59]:
len(documents)

375

In [60]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)

In [61]:
relevance_total = []

for q in tqdm(ground_truth):
  
    doc_id = q['document']
    qs= q["question"]
    v = np.array(embedding_model.encode(qs))
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)


100%|██████████| 1830/1830 [01:59<00:00, 15.37it/s]


In [62]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [63]:
hit_rate(relevance_total)

0.9398907103825137

## Q5

In [64]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [65]:
for doc, embedding in zip(tqdm(documents), embeddings):
    doc["question_text_vector"] = embedding


100%|██████████| 375/375 [00:00<00:00, 714288.83it/s]


In [66]:
len(documents)

375

In [67]:
len(documents)

375

In [68]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:02<00:00, 125.62it/s]


In [69]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,

        }
    

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [70]:
f = "question_text_vector"


In [71]:
elastic_search_knn(field=f, vector=user_vector)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [72]:
user_question

'I just discovered the course. Can I still join it?'

## Q6

In [73]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [74]:
relevance_total2 = []

for q in tqdm(ground_truth):
  
    doc_id = q['document']
    qs= q["question"]
    v = np.array(embedding_model.encode(qs))
    results = elastic_search_knn(field=f, vector=v)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total2.append(relevance)

100%|██████████| 1830/1830 [00:48<00:00, 37.54it/s]


In [75]:
def hit_rate(relevance_total2):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total2)

In [76]:
hit_rate(relevance_total2)

0.9398907103825137